In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import gzip
import sys
import struct
import urllib.request

def read_image(fi):
    magic, n, rows, columns = struct.unpack(">IIII", fi.read(16))
    assert magic == 0x00000803
    assert rows == 28
    assert columns == 28
    rawbuffer = fi.read()
    assert len(rawbuffer) == n * rows * columns
    rawdata = np.frombuffer(rawbuffer, dtype='>u1', count=n*rows*columns)
    return rawdata.reshape(n, rows, columns).astype(np.float32) / 255.0

def read_label(fi):
    magic, n = struct.unpack(">II", fi.read(8))
    assert magic == 0x00000801
    rawbuffer = fi.read()
    assert len(rawbuffer) == n
    return np.frombuffer(rawbuffer, dtype='>u1', count=n)

def openurl_gzip(url):
    request = urllib.request.Request(
        url,
        headers={
            "Accept-Encoding": "gzip",
            "User-Agent": "Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11", 
        })
    response = urllib.request.urlopen(request)
    return gzip.GzipFile(fileobj=response, mode='rb')

if __name__ == '__main__':
    np.savez_compressed(
        'mnist',
        train_x=read_image(openurl_gzip('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz')),
        train_y=read_label(openurl_gzip('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz')),
        test_x=read_image(openurl_gzip('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz')),
        test_y=read_label(openurl_gzip('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'))
    )

In [3]:
data = np.load('mnist.npz')

print("Training data (X):", data['train_x'].shape, data['train_x'].dtype)
print("Training data (Y):", data['train_y'].shape, data['train_y'].dtype)
print("Test data (X):", data['test_x'].shape, data['test_x'].dtype)
print("Test data (Y):", data['test_y'].shape, data['test_y'].dtype)

Training data (X): (60000, 28, 28) float32
Training data (Y): (60000,) uint8
Test data (X): (10000, 28, 28) float32
Test data (Y): (10000,) uint8


In [4]:
def images_to_vectors(X):
    X = np.reshape(X, (len(X), -1))         # Flatten: (N x 28 x 28) -> (N x 784)
    return np.c_[X, np.ones(len(X))]        # Append 1: (N x 784) -> (N x 785)

In [55]:
X_train = images_to_vectors(data['train_x'])
Y_train = data['train_y']                        
X_test = images_to_vectors(data['test_x'])
Y_test = data['test_y']

In [125]:
# Q1
def softmax(a):
    ea = np.exp(a - np.max(a))
    return ea / ea.sum()

def calculate_LDA(max_epochs, eta0):
  W = np.zeros((10, X_train.shape[1]))
  for t in range(max_epochs):
    eta = eta0 / np.sqrt(1+t)
    j = np.random.randint(0, X_train.shape[0])
    a = np.dot(W, X_train[j])
    p = softmax(a)
    Y = np.zeros(10)
    Y[Y_train[j]] = 1 
    W = np.array([W[k] + eta * (Y[k] - p[k]) * X_train[j] for k in range(10)] )
  return W

In [137]:
# Q2
W_ep2_e05 = calculate_LDA(100, 0.5)
W_ep3_e05 = calculate_LDA(1000, 0.5)
W_ep4_e05 = calculate_LDA(10000, 0.5)
W_ep5_e05 = calculate_LDA(100000, 0.5)
W_ep6_e05 = calculate_LDA(1000000, 0.5)


N = X_test.shape[0]

def check_model(W):
  count = 0

  for i in range(N):
    a = np.dot(W, X_test[i])
    y_hat = np.where(a == max(a))
    if Y_test[i] == y_hat:
      count += 1
  return count / N

print('W_ep2_e05:', check_model(W_ep2_e05))
print('W_ep3_e05:', check_model(W_ep3_e05))
print('W_ep4_e05:', check_model(W_ep4_e05))
print('W_ep5_e05:', check_model(W_ep5_e05))
print('W_ep6_e05:', check_model(W_ep6_e05))

W_ep2_e05: 0.55
W_ep3_e05: 0.825
W_ep4_e05: 0.8956
W_ep5_e05: 0.9154
W_ep6_e05: 0.9221
